## Installation

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(f"Vlasta/DNA_Sentencepiece_vocab_10000_max_tokenlen_45")

Downloading:   0%|          | 0.00/257 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
from datasets import load_dataset

splitted_datasets = load_dataset("simecek/Human_DNA_v0")

Using custom data configuration simecek--Human_DNA_v0-3127ba11a87ac1a1
Reusing dataset parquet (/home/jovyan/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0-3127ba11a87ac1a1/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
tokenizer("ATGGAAAGAGGCACCATTCT")    

{'input_ids': [2, 33, 1246, 2031, 6, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [13]:
def tokenize_function(s):
  return tokenizer(s['Seq'])

tokenize_function({'Seq':'ACCTGCTGGACGATCATA'})

{'input_ids': [2, 265, 8749, 411, 29995, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [14]:
tokenized_datasets = splitted_datasets.map(tokenize_function, remove_columns='Seq', num_proc=4)
tokenized_datasets

#0:   0%|          | 0/7324 [00:00<?, ?ex/s]

#1:   0%|          | 0/7324 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1517 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1575 > 512). Running this sequence through the model will result in indexing errors


#3:   0%|          | 0/7324 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1624 > 512). Running this sequence through the model will result in indexing errors


...

#2:   0%|          | 0/65915 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1599 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1553 > 512). Running this sequence through the model will result in indexing errors


#3:   0%|          | 0/65914 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1381 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 29296
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 263659
    })
})

In [15]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

def group_texts(examples, max_length=512):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

chunked_datasets = tokenized_datasets.map(group_texts, batched=True, desc=f"Grouping texts in chunks of 512")
chunked_datasets

Grouping texts in chunks of 512:   0%|          | 0/30 [00:00<?, ?ba/s]

Grouping texts in chunks of 512:   0%|          | 0/264 [00:00<?, ?ba/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 87527
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 787698
    })
})

In [16]:
chunked_datasets.push_to_hub("Human_DNA_v0_SentencepieceTokenized_vocab10k")

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/5 [00:00<?, ?it/s]